<a href="https://colab.research.google.com/github/datvodinh10/recurrent-ppo/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!lscpu | grep 'Model name'

'lscpu' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!git clone https://github.com/datvodinh10/recurrent-ppo.git
%cd recurrent-ppo

^C
c:\Users\vodin\recurrent-ppo\recurrent-ppo


In [ ]:
from model.trainer import *
%cd ENV
import env

In [ ]:
game_name = "Splendor_v3"
env.make(game_name)
writer_path = f"/content/"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
save_path = f"/content/gdrive/MyDrive/Data-PPO-LSTM/{game_name}/"

In [ ]:
import numpy as np
from numba import njit

def bot_max_eps_length(state, perData):
    validActions = env.getValidActions(state)
    arr_action = np.where(validActions == 1)[0]
    idx = np.random.randint(0, arr_action.shape[0])
    perData[0]+=1
    if env.getReward(state)!=-1:
        if perData[0] > perData[1]:
            perData[1] = perData[0]
        perData[0] = 0
    return arr_action[idx], perData
per = np.array([0,0])
env.run(bot_max_eps_length,200,per,1)
max_eps_length = per[1]
max_eps_length

In [ ]:
config = {
    "PPO":{
        "critic_coef": 1,
        "policy_kl_range":0.0008,
        "policy_params": 20,
        "gamma":0.99,
        "gae_lambda":0.95,
        "value_clip": 0.2,

    },
    "LSTM":{
        "max_eps_length":max_eps_length + 50,
        "seq_length":max_eps_length if max_eps_length < 128 else 128,
        "hidden_size":64,
        "embed_size": 64,
    },
    "entropy_coef":{
        "start": 0.01,
        "end": 0.001,
        "step": 100_000
    },
    "lr":2.5e-4,
    "num_epochs": 2,
    "num_game_per_batch":128,
    "max_grad_norm": 0.5,
    "n_mini_batch": 4,
    "rewards": [0,1,-0.001], # [lose,win,not_done]
    "set_detect_anomaly": True,
    "normalize_advantage": True,
    "encode_state": False
}


In [ ]:
trainer = Trainer(config=config,env=env,writer_path = writer_path,save_path=save_path)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/

In [ ]:
trainer.train()